<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0526onomatope_test_newsents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
HOME = '/Users/asakawa'

import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install --upgrade transformers
    !pip install --upgrade 'fugashi[ipadic]'
    !pip install --upgrade ipadic
    
    !pip install --upgrade termcolor
    !pip install --upgrade jaconv


In [ ]:
from transformers import BertJapaneseTokenizer
from transformers import BertForMaskedLM
import torch
import transformers

# stockmarket 本での事前訓練済データ
# model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# model_ja_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
model_ja_name = 'cl-tohoku/bert-base-japanese'  # 東北大学乾研による 日本語 BERT 実装
# see https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2
#model_ja_name = 'sonoisa/sentence-bert-base-ja-mean-tokens-v2'  # 東北大学乾研による 日本語 BERT 実装

tknz = BertJapaneseTokenizer.from_pretrained(model_ja_name)
model_onmt = BertForMaskedLM.from_pretrained(model_ja_name, return_dict = True)
model_orig = BertForMaskedLM.from_pretrained(model_ja_name, return_dict = True)

# リソースの選択（CPU/GPU）
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequen

In [ ]:
# 2021/Jan 近藤先生からいただいたオノマトペ辞典のデータの読み込み
import os
from termcolor import colored

#'日本語オノマトペ辞典4500より.xls' は著作権の問題があり，公にできません。
# そのため Google Colab での解法，ローカルファイルよりアップロードしてください
if isColab:
    from google.colab import files
    uploaded = files.upload()  # ここで `日本語オノマトペ辞典4500より.xls` を指定してアップロードする
    data_dir = '.'
else:
    data_dir = os.path.join(HOME, 'study/2021ccap/notebooks')

import pandas as pd
import jaconv

onomatopea_excel = '2021-0325日本語オノマトペ辞典4500より.xls'
onmtp2761 = pd.read_excel(os.path.join(data_dir, onomatopea_excel), sheet_name='2761語')

#すべてカタカナ表記にしてデータとして利用する場合
#`日本語オノマトペ辞典4500` はすべてひらがな表記だが，一般にオノマトペはカタカナ表記されることが多いはず
#onomatopea = list(sorted(set([jaconv.hira2kata(o) for o in onmtp2761['オノマトペ']])))

# Mac と Windows の表記の相違を吸収
onomatopea = list(sorted(set([jaconv.normalize(o) for o in onmtp2761['オノマトペ']])))
print(f'データファイル名: {os.path.join(data_dir, onomatopea_excel)}\n',
      f'オノマトペ単語総数: len(onomatopea):{len(onomatopea)}')

# トークナイザ の修正，実際には onomatopea 単語リストを引数に指定して `add_tokens()` を呼び出すだけ
# ただし，語彙数 tknz.vocab は変更されない。追加された語彙，本コードの場合はオノマトペは，
# `tknz.added_tokens_encoder` と `tknz1.added_tokens_decoder` に反映されているためである
num_added = tknz.add_tokens(onomatopea)
print(f'追加されたトークン数:{num_added}/オノマトペ数:{len(onomatopea)}') 
model_onmt.resize_token_embeddings(len(tknz))
model_orig.resize_token_embeddings(len(tknz))

print(f' len(tknz):{len(tknz)}\n', 
      f'len(tknz.vocab):{len(tknz.vocab)}\n',  # 一見すると，この数字からオノマトペが追加されていないように見える。
      f'tknz.vocab_size:{tknz.vocab_size}')    # 駄菓子菓子，下で見るように，正しく動作しているように見受けられる

print('# 確認用')
for w in onomatopea[-5:]:
    idx = tknz.convert_tokens_to_ids(w)
    w_ = tknz.convert_ids_to_tokens(idx)
    print(f'単語:{w}(id:{idx}) -> token:{w_}')
    

データファイル名: /Users/asakawa/study/2021ccap/notebooks/2021-0325日本語オノマトペ辞典4500より.xls
 オノマトペ単語総数: len(onomatopea):1741
追加されたトークン数:1711/オノマトペ数:1741
 len(tknz):33711
 len(tknz.vocab):32000
 tknz.vocab_size:32000
# 確認用
単語:わんわ(id:33706) -> token:わんわ
単語:わんわん(id:33707) -> token:わんわん
単語:わーっ(id:33708) -> token:わーっ
単語:わーわー(id:33709) -> token:わーわー
単語:わーん(id:33710) -> token:わーん


In [ ]:
# 訓練済モデルの読み込み
model_onmt.load_state_dict(torch.load('2022_0125onomatopea.pt'))

<All keys matched successfully>

In [ ]:
from torch.nn import functional as F

_newsents = ['夏の暑さが肌に[MASK]感じる。',
             '真夏の太陽が[MASK]と照り付ける。',
             '勢いよく雨が[MASK]と降る。',
             'ろうそくの火が消えそうに[MASK]灯る。',
             '砂糖がこぼれて[MASK]落ちる。',
             '水が沸騰して[MASK]沸く。',
             'お風呂で[MASK]と温まる。',
             'どんぶりに汁を[MASK]と注ぐ。',
             '日が照り[MASK]と心地がよい。',
             'クーラーから[MASK]と風が吹く。',
             '髪の毛が[MASK]と燃える。',
             'お酒に酔ってちどり足で[MASK]歩く。',
             '犬が顔を[MASK]舐める。',
             '風邪で咳が[MASK]出る',
             'けがをした子供が[MASK]と泣く',
             'ふくれっ面で[MASK]怒る',
             '悲しいことを思い出して、[MASK]する',
             '昼寝や休暇を取って、[MASK]する',
             'スポーツ選手のような[MASK]とした体だ',
             '物事が順調にいき調子よく[MASK]進む',
             '聞こえずらい小声で[MASK]話す',
             '水の中を平泳ぎで[MASK]進む',
             '翼を大きく[MASK]と広げる',
             '腕を回して肩を[MASK]と音を鳴らす',
             '地震が起きて建物が[MASK]と揺れる',
             '包丁で指を[MASK]と切る',
             '皿を擦り[MASK]と鳴らす',
             '電車が音を鳴らし[MASK]と走る',
             '花が枯れてしまい[MASK]する',
             '雑巾を[MASK]と絞る',
             '探偵が見つからないように[MASK]動き回る',
             '分散してみんな[MASK]に進む',
             '宝石を磨いてきれいに[MASK]光る',
             '髭が中途半端に生えて[MASK]した肌ざわりだ',
             '頭が割れるように[MASK]痛む', 
             '花火が音をたて打ち上がり[MASK]と爆ぜる',
             '魚が打ちあがり勢いよく[MASK]動く', 
             '揚げ物が軽快に[MASK]と揚がる',
             '突然空が雨雲に代わり雨が[MASK]と降り出した。',
             '風が通り抜け音を立てて[MASK]と鳴る',
             '人が小走りで道の上を[MASK]歩く',
             '祭りで和太鼓を[MASK]叩く',
             '虫がすばやく目の前を[MASK]と飛ぶ',
             '相撲取りがしりもちを[MASK]とつく',
             '友達に悪口を言われて[MASK]する',
             '急に雨が降り[MASK]と服が濡れる',
             '丸々太ったあざらしが坂を[MASK]転がる',
             '寝起きで寝癖がつき髪が[MASK]している',
             '驚いて口を[MASK]と大きく開け驚く',
             'ゴミ箱の中身を[MASK]音を立て漁る',
             '蛍光灯が切れかけて[MASK]と点滅する',
             '木が鈍く音をたてて[MASK]と倒れる',
             '硬そうなとても大きな角ばった[MASK]とした岩',
             'とても滑らかな光沢のある水晶が[MASK]と輝いている',
             '煎餅をたべ食べかすが[MASK]と崩れ落ちる',
             '熱い鉄板に水てきが落ちて[MASK]と蒸発する',
             '上司にしつこく怒られ[MASK]と説教される',
             '人に聞かれたくない噂話を友達と小さな声で[MASK]と話す',
             '街中でお祭りがあり人が集まり[MASK]と騒がしい',
             '何もすることがなく[MASK]と街中を歩く',
             '温泉の中は湯気が立ち込めて[MASK]と霧がかる',
             'シャツの汚れを取るために[MASK]と擦る',
             '重い鐘が重く[MASK]と鳴る', 
             '牛が遅い速度で[MASK]歩き', 
             '口を開けて[MASK]と音を立てて食べる',
             '炎が赤く勢いよく[MASK]と燃える',
             '誕生日に思いがけないプレゼントをもらい[MASK]する',
             '竹がいつの間にか土の中から[MASK]と育つ', 
             '風が吹きカーテンを[MASK]と揺らす', 
             '花瓶を割ったことがバレそうになり[MASK]とする', 
             '厳しい指導を受け[MASK]と弱音を吐く', 
             '発表結果が待ちきれず[MASK]と落ち着かない', 
             'バケツを倒してしまい水が[MASK]と一気に出る', 
             '浮き輪が水面に[MASK]と揺れ浮く', 
             '細い折れ木の先で動物を[MASK]と突く', 
             '冷たい飲み物を喉を鳴らし[MASK]音を立てながら飲む', 
             '小さな子供がうずくまって[MASK]と泣く', 
             '温かい部屋の中で[MASK]と怠惰に過ごす', 
             '熟睡し大きないびきを[MASK]と立てる', 
             '水が紙に染み込んで[MASK]と跡をつくる', 
             'いらない紙を手で握り潰してく[MASK]と丸める', 
             'ろうそくを消すために口をすぼめて[MASK]と息をかける', 
             '高貴な婦人が上品に[MASK]と高笑う', 
             '鋭い表情で[MASK]とした顔立ちだ', 
             '売れ行きがよくお金が大量に[MASK]と儲かる', 
             '冬の乾燥した晴れた空気が[MASK]と心地よい', 
             '早急にしなければならないことが見つかり[MASK]と忙しない', 
             '品がなく大声で不良が[MASK]と笑う', 
             '粘着性のある湿布が肌に[MASK]とくっつく', 
             '金槌で金属を叩き硬い音で[MASK]と鳴る', 
             '木箱に入った何かが揺れて[MASK]と音を立てる', 
             '先生とすれ違い挨拶の為に[MASK]とお辞儀をした', 
             'スリッパが音をたて[MASK]と軽く弾む', 
             '大きな袋の中身はたくさんのものが入り[MASK]と重い', 
             'プールで遊ぶ子供が[MASK]と水をかけあう', 
             'はさみで前髪を躊躇なく[MASK]と切る',
             '断りずらい頼みごとをされたので仕方なく[MASK]と断る', 
             '疲れ切った男はしわの入った[MASK]としたスーツを着ている', 
             '勢いよく廊下を走り抜け[MASK]と音を立てる',
             '立派な男になった彼は[MASK]したスーツを着て出社する'
            ]

N = 5
for i, text in enumerate(_newsents):
    print(colored(f'{i:3d} text:{text}', color='blue', attrs=['bold']))
    inputs = tknz.encode_plus(text, return_tensors = "pt")
    mask_index = torch.where(inputs['input_ids'][0] == tknz.mask_token_id)
    outputs  = model_onmt(**inputs)
    outputs0 = model_orig(**inputs)
    logits = outputs.logits
    logits0 = outputs0.logits
    softmax = F.softmax(logits, dim=-1)
    softmax0 = F.softmax(logits0, dim=-1)
    mask_word = softmax[0, mask_index, :]
    mask_word0 = softmax0[0, mask_index, :]
    topN = torch.topk(mask_word, N, dim=1)[1][0]
    topN0 = torch.topk(mask_word0, N, dim=1)[1][0]

    for i, (token, token0) in enumerate(zip(topN, topN0)):
        wrd = tknz.convert_ids_to_tokens([token])
        wrd0 = tknz.convert_ids_to_tokens([token0])
        sentence_replaced = text.replace(tknz.mask_token, wrd[0])
        print(f'{i+1:2d}', colored(sentence_replaced, color='grey', attrs=['bold']), end=" ")
        sentence_replaced0 = text.replace(tknz.mask_token, wrd0[0])
        print(colored(sentence_replaced0, color='red', attrs=['bold']))


  0 text:夏の暑さが肌に[MASK]感じる。
 1 夏の暑さが肌にも感じる。 夏の暑さが肌にも感じる。
 2 夏の暑さが肌にと感じる。 夏の暑さが肌には感じる。
 3 夏の暑さが肌に[UNK]感じる。 夏の暑さが肌に優しく感じる。
 4 夏の暑さが肌にどっと感じる。 夏の暑さが肌に強く感じる。
 5 夏の暑さが肌にこう感じる。 夏の暑さが肌によく感じる。
  1 text:真夏の太陽が[MASK]と照り付ける。
 1 真夏の太陽が春と照り付ける。 真夏の太陽がゆっくりと照り付ける。
 2 真夏の太陽がさーっと照り付ける。 真夏の太陽がはっきりと照り付ける。
 3 真夏の太陽が度と照り付ける。 真夏の太陽が次々と照り付ける。
 4 真夏の太陽がかしゃっと照り付ける。 真夏の太陽が[UNK]と照り付ける。
 5 真夏の太陽が、と照り付ける。 真夏の太陽がしっかりと照り付ける。
  2 text:勢いよく雨が[MASK]と降る。
 1 勢いよく雨がはらっと降る。 勢いよく雨が降ると降る。
 2 勢いよく雨がどすんと降る。 勢いよく雨がゆっくりと降る。
 3 勢いよく雨がぽつりと降る。 勢いよく雨が次々と降る。
 4 勢いよく雨がどーっと降る。 勢いよく雨がはっきりと降る。
 5 勢いよく雨ががばっと降る。 勢いよく雨が堂々と降る。
  3 text:ろうそくの火が消えそうに[MASK]灯る。
 1 ろうそくの火が消えそうにふっくら灯る。 ろうそくの火が消えそうになく灯る。
 2 ろうそくの火が消えそうに赤く灯る。 ろうそくの火が消えそうになり灯る。
 3 ろうそくの火が消えそうにどっと灯る。 ろうそくの火が消えそうにまで灯る。
 4 ろうそくの火が消えそうにびたり灯る。 ろうそくの火が消えそうにも灯る。
 5 ろうそくの火が消えそうに終日灯る。 ろうそくの火が消えそうに赤く灯る。
  4 text:砂糖がこぼれて[MASK]落ちる。
 1 砂糖がこぼれて高く落ちる。 砂糖がこぼれても落ちる。
 2 砂糖がこぼれてぽつり落ちる。 砂糖がこぼれて溶け落ちる。
 3 砂糖がこぼれてががっ落ちる。 砂糖がこぼれて舞い落ちる。
 4 砂糖がこぼれて爆発落ちる。 砂糖がこぼれてすぐ落ちる。
 5 砂糖がこぼれてすぐ落ちる。 砂糖がこぼれてから落ちる。
  5 text:水が沸